In [11]:
import subprocess
import json
import requests
import pdb

In [9]:
def get_region(SNP_ids):
    """
    Given a list SNPs id, extract the region that encompassed them all from the ENSEMBL REST API.
    Advised only for small SNP set sizes < 10.
    """
    
    SNP_json = json.dumps({"ids": SNP_ids})
    server = "http://rest.ensembl.org"
    ext = "/variation/homo_sapiens"
    headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
    r = requests.post(server+ext, headers=headers, data=SNP_json)

    if not r.ok:
        print "Regions of one of more SNPs could not be retieved"
        r.raise_for_status()
        sys.exit()

    decoded = r.json()

    decoded.keys()

    sorted_locations = sorted([decoded[k]['mappings'][0]['location'] for k in decoded.keys()])
    print sorted_locations

    region = '-'.join([sorted_locations[0].split('-')[0],sorted_locations[-1].split('-')[1]])

    return region

In [103]:
def calculate_LD_window(SNP_id, window_size):

    loc = get_region([SNP_id])

    from_pos = int(loc.split(":")[1].split('-')[0]) - (window_size / 2)
    to_pos = int(loc.split(":")[1].split('-')[1]) + (window_size / 2)

    chromosome = loc.split(':')[0]
    print from_pos
    print to_pos


    region = '{}:{}-{}'.format(chromosome,from_pos,to_pos)
    
    extract_region_comm = "bcftools view -r {} ../data/processed/CEPH.chr{}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.nodup.bcf.gz -O v -o region.vcf".format(region,chromosome)

    subprocess.call(extract_region_comm.split(" "))

    region_file = open('region.vcf','r')
    region_vcf = region_file.read()
    
    SNPs_order = re.findall('rs[0-9]+', region_vcf)

    plinkcomm = "plink2 --vcf region.vcf --r square --out testLDwindow"
    plinkcomm_list = plinkcomm.split(" ")
    subprocess.call(plinkcomm_list)
    
    LD_file = open('testLDwindow.ld','r')
    g = LD_file.read()
    LD_array = [x.split('\t') for x in g.splitlines()]
    LD_file.close
    
    
    return SNPs_order, LD_array



In [108]:
%%time
res = calculate_LD_window('rs74509095', 10000)

[u'1:7851569-7851569']
7846569
7856569
CPU times: user 12.4 ms, sys: 9.06 ms, total: 21.5 ms
Wall time: 136 ms


In [101]:
### Needs list of SNPs in a file, aswell as which chromosome they are from.
### Errors to catch: some SNPs not in this chromosome, deduplicate step
### modify to BCFtools, 

def calculate_pairwise_LD(SNPs_filepath=None,SNP_ids=None, region=None,db=0):
    """
    For large numbers of SNPs, best to 0specify SNP region with chrom:to-from, e.g. 1:7654947-8155562
    For small numbers (<10), regions are extracted from ENSEMBL REST API.
    SNPs can be inputted in a list or from a file with one SNP id per line.
    """

    assert SNPs_filepath or SNP_ids, "SNPs must be inputted either from a file or a list"
    
    ## If a SNP file is provided, use it. Otherwise continue with the provided SNP ids.
    if SNPs_filepath:
        SNPs_file = open(SNPs_filepath, 'r')
        SNP_ids = SNPs_file.read().splitlines()
        SNPs_file.close()

    

    ##If a region is not specified, extract it using ENSEMBL REST API. If large amount of SNPs, advise specifying region.
    if not region: 
        region = get_region(SNP_ids)
    
    if db != 1:
        print region
        
    
 
    chromosome = region.split(':')[0]

    extract_region_comm = "bcftools view -r {} ../data/processed/CEPH.chr{}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.nodup.bcf.gz -O z -o region.vcf.gz".format(region,chromosome)

    subprocess.call(extract_region_comm.split(" "))

    region_file = "region.vcf.gz"
    ####

    vcfcomm = "vcftools --gzvcf {} --snps {} --recode --stdout".format(region_file, SNPs_filepath)


    vcf = subprocess.check_output(vcfcomm.split(" "))

    if db != 1:
        subprocess.call(['rm', 'region.vcf.gz'])


    f = open('temp.vcf', 'w')
    f.write(vcf)
    f.close()
    
    SNPs_order = re.findall('rs[0-9]+', vcf)
    

    plinkcomm = "plink2 --vcf temp.vcf --r square --out testLD"
    plinkcomm_list = plinkcomm.split(" ")
    subprocess.call(plinkcomm_list)
    
    if db != 1:
        subprocess.call(['rm', 'temp.vcf'])
    
    LD_file = open('testLD.ld','r')
    g = LD_file.read()
    LD_array = [x.split('\t') for x in g.splitlines()]
    LD_file.close
    
    if db != 1:
        subprocess.call(['rm', 'temp.vcf'])
    
    return (SNPs_order, LD_array)
    


In [109]:
time calculate_pairwise_LD("../data/raw/smallrsIDs.txt", db=0)

[u'1:7637119-7637119', u'1:7645031-7645031', u'1:7819141-7819141', u'1:7851569-7851569', u'1:7866341-7866341', u'1:7878053-7878053', u'1:7968778-7968778']
1:7637119-7968778
CPU times: user 6.98 ms, sys: 12.5 ms, total: 19.5 ms
Wall time: 421 ms


(['rs6661496',
  'rs79544751',
  'rs148043253',
  'rs74509095',
  'rs4908705',
  'rs4908708'],
 [['1', '1', '-0.0256579', '-0.0413742', '-0.0729325', '-0.0729325'],
  ['1', '1', '-0.0256579', '-0.0413742', '-0.0729325', '-0.0729325'],
  ['-0.0256579', '-0.0256579', '1', '0.325107', '0.248882', '0.248882'],
  ['-0.0413742', '-0.0413742', '0.325107', '1', '-0.0778898', '-0.0778898'],
  ['-0.0729325', '-0.0729325', '0.248882', '-0.0778898', '1', '1'],
  ['-0.0729325', '-0.0729325', '0.248882', '-0.0778898', '1', '1']])